In [6]:
# third-party
import pandas as pd
import numpy as np
import pickle

# local
from waveform import get_waveforms_by_id_by_activity
from get_data import get_data_by_id_activity
from constants import DEVICE_INFO

In [7]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# # remove middle activities
# for id in overview.keys():
#     del overview[id]["SNBm"]
#     del overview[id]["UALm"]
#     del overview[id]["UARm"]

acquisition_folderpath = 'Aquisicao'
data, _ = get_data_by_id_activity(acquisition_folderpath)

Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [8]:
amplitudes_df = pd.DataFrame(columns=['amplitude', 'activity', 'sensor', 'id'])

for id in overview.keys():
    for activity in overview[id].keys():
        try:
            for device in ['mag', 'pzt']:
                waveforms,_ = get_waveforms_by_id_by_activity(data[id][activity][device].values, overview[id][activity][DEVICE_INFO[device]["name"]])
                x = np.amax(waveforms, axis=1) - np.amin(waveforms, axis=1)
                amplitudes_df = pd.concat([amplitudes_df, pd.DataFrame({'amplitude': x, 'activity': activity, 'sensor': DEVICE_INFO[device]["sensor"], 'id': id})], ignore_index=True)
        except KeyError as e:
            print(f"Coundn't find key {e} for id {id}")

amplitudes_df            

Coundn't find key 'MIXB' for id QMQ7


,amplitude,activity,sensor,id
0,40.535129,SNB,MAG,7OYX
1,27.815240,SNB,MAG,7OYX
2,40.000312,SNB,MAG,7OYX
3,46.846376,SNB,MAG,7OYX
4,44.401724,SNB,MAG,7OYX
...,...,...,...,...
3638,2001.090157,TR,PZT,D4GQ
3639,2665.469705,TR,PZT,D4GQ
3640,2999.067703,TR,PZT,D4GQ
3641,1208.848368,TR,PZT,D4GQ


In [9]:
amplitudes_df[amplitudes_df["sensor"]=='MAG'].max()

amplitude    330.553946
activity            UAR
sensor              MAG
id                 Y6O3
dtype: object

In [10]:
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG'])} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG'])}")
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT'])} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT'])}")

print(np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']))
print(np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']))

pk-to-pk for MAG sensor: 56.799463131981476 +/- 32.26809251306975
pk-to-pk for MAG sensor: 773.3250611483963 +/- 692.9108827411218
0.5681055899787352
0.896015036305873


In [11]:
amplitudes_by_activity_df = pd.DataFrame(columns=['Activity', 'Sensor', 'Pk-to-pk amplitude', 'std/mean'])

for activity in amplitudes_df['activity'].unique():
    amps = amplitudes_df[(amplitudes_df['activity'] == activity) & (amplitudes_df['sensor'] == 'MAG')]['amplitude'].values
    mean_amp = np.mean(amps)
    std_amp = np.std(amps)
    amplitudes_by_activity_df = pd.concat([amplitudes_by_activity_df, pd.DataFrame({'Activity': activity, 'Sensor': 'MAG', 'Pk-to-pk amplitude': f'${np.around(mean_amp, 2)} \pm {np.around(std_amp)}$', 'std/mean': np.round(std_amp/mean_amp, 2)}, index=[0])], ignore_index=True)
    
    amps = amplitudes_df[(amplitudes_df['activity'] == activity) & (amplitudes_df['sensor'] == 'PZT')]['amplitude'].values
    mean_amp = np.mean(amps)
    std_amp = np.std(amps)
    amplitudes_by_activity_df = pd.concat([amplitudes_by_activity_df, pd.DataFrame({'Activity': activity, 'Sensor': 'PZT', 'Pk-to-pk amplitude': f'${np.around(mean_amp, 2)} \pm {np.around(std_amp)}$', 'std/mean': np.round(std_amp/mean_amp, 2)}, index=[0])], ignore_index=True)

amplitudes_by_activity_df

,Activity,Sensor,Pk-to-pk amplitude,std/mean
0,SNB,MAG,$35.68 \pm 14.0$,0.41
1,SNB,PZT,$751.42 \pm 570.0$,0.76
2,SGB,MAG,$75.28 \pm 37.0$,0.49
3,SGB,PZT,$1504.64 \pm 947.0$,0.63
4,MCH,MAG,$57.9 \pm 26.0$,0.45
5,MCH,PZT,$757.01 \pm 511.0$,0.68
6,SQT,MAG,$72.34 \pm 38.0$,0.53
7,SQT,PZT,$908.98 \pm 600.0$,0.66
8,MIXB,MAG,$50.76 \pm 28.0$,0.55
9,MIXB,PZT,$1186.04 \pm 713.0$,0.60


In [12]:
amplitudes_df

,amplitude,activity,sensor,id
0,40.535129,SNB,MAG,7OYX
1,27.815240,SNB,MAG,7OYX
2,40.000312,SNB,MAG,7OYX
3,46.846376,SNB,MAG,7OYX
4,44.401724,SNB,MAG,7OYX
...,...,...,...,...
3638,2001.090157,TR,PZT,D4GQ
3639,2665.469705,TR,PZT,D4GQ
3640,2999.067703,TR,PZT,D4GQ
3641,1208.848368,TR,PZT,D4GQ


### Analysis on alternative positioning

In [13]:
amplitudes_middle_df = pd.DataFrame(columns=['amplitude', 'activity', 'sensor', 'id'])

for id in overview_middle.keys():
    for activity in overview_middle[id].keys():
        for device in ['mag', 'pzt']:
            waveforms,_ = get_waveforms_by_id_by_activity(data[id][activity][device].values, overview_middle[id][activity][DEVICE_INFO[device]["name"]])
            x = np.amax(waveforms, axis=1) - np.amin(waveforms, axis=1)
            amplitudes_middle_df = pd.concat([amplitudes_middle_df, pd.DataFrame({'amplitude': x, 'activity': activity, 'sensor': DEVICE_INFO[device]["sensor"], 'id': id})], ignore_index=True)

amplitudes_middle_df            

,amplitude,activity,sensor,id


In [14]:
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG'])} +/- {np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG'])}")
print(np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG']) / np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG']))

pk-to-pk for MAG sensor: nan +/- nan
nan
